In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [26]:
## Function to extract the details
#----------------------------------------------
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available    

In [32]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

# The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"
    
    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
# Store the links
    links_list = []

 # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    amazon_df = pd.DataFrame.from_dict(d)
    ## there is some null values are present inside the title
    ## we replace these blank values as nan
    amazon_df['title'].replace('', np.nan, inplace=True)
    ## we wnat to remove the products those who don;t have the title name by using the dropna statement
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


ConnectionError: HTTPSConnectionPool(host='www.amazon.comhttps', port=443): Max retries exceeded with url: /aax-us-iad.amazon.com/x/c/JCLtoFqz5meZGWnNgY6D9koAAAGQTi9zUgEAAAH2AQBvbm9fdHhuX2JpZDEgICBvbm9fdHhuX2ltcDEgICCNh7ZX/https://www.amazon.com/AceGamer-Light-up-Controller-Compatible-PlayStation-4/dp/B0C8J3R84H/ref=sxbs_sbv_search_btf?content-id=amzn1.sym.2f0a8989-0b67-47e7-b61e-9e3ef9908602%3Aamzn1.sym.2f0a8989-0b67-47e7-b61e-9e3ef9908602&cv_ct_cx=playstation+4&dib=eyJ2IjoiMSJ9.smrgQsrI2rApjAETXyjf5A.9rJlK_5AqR9f60oUr3YVsdV7_9ksbUnPeKvYFgD5EmE&dib_tag=se&keywords=playstation+4&pd_rd_i=B0C8J3R84H&pd_rd_r=40650ce9-760c-46e1-8c75-b06a17858b49&pd_rd_w=2SvSN&pd_rd_wg=7pTsj&pf_rd_p=2f0a8989-0b67-47e7-b61e-9e3ef9908602&pf_rd_r=PH7XM9Q9C8CXPA7H0QWQ&qid=1719298650&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sr=1-1-a61ee601-6e56-4862-a8a2-1d3da5a5406f (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000022C483A6FC0>: Failed to resolve 'www.amazon.comhttps' ([Errno 11

In [ ]:
amazon_df

In [ ]:
links[0].get("href")

In [15]:
links_list

['/PlayStation%C2%AE4-Console-Modern-Warfare-PlayStation-4/dp/B0BP3ZK9K9/ref=sr_1_1?dib=eyJ2IjoiMSJ9.4nmDhjd15fhApzou_N5hFAWCpuRt0BhU8x7lOv51On8lJBf5SlXv1VqpB4CHwgZW8MONP_oVyHbGlkrW2VqIDjy0lJ0GoyuqyKtjamj8C6aKdVR7vwXXdxV62dhLr2nQnd14VHmjSfJfxTQJc3TWsh4E8oiCxtQ_6zjMQrgmGxh2VTPis19z7SDhd4DnW_075TXzpjogRNwKvbT5n-vJoJzAHUtcL98PDinuRaHwI1w.baaNvO_szBcoLzW1NpiwP18YEIDaRk2Dti1DyQTRZdU&dib_tag=se&keywords=playstation+4&qid=1719296832&sr=8-1',
 '/Sony-Playstation-PS4-Black-Console/dp/B012CZ41ZA/ref=sr_1_2_mod_primary_new?dib=eyJ2IjoiMSJ9.4nmDhjd15fhApzou_N5hFAWCpuRt0BhU8x7lOv51On8lJBf5SlXv1VqpB4CHwgZW8MONP_oVyHbGlkrW2VqIDjy0lJ0GoyuqyKtjamj8C6aKdVR7vwXXdxV62dhLr2nQnd14VHmjSfJfxTQJc3TWsh4E8oiCxtQ_6zjMQrgmGxh2VTPis19z7SDhd4DnW_075TXzpjogRNwKvbT5n-vJoJzAHUtcL98PDinuRaHwI1w.baaNvO_szBcoLzW1NpiwP18YEIDaRk2Dti1DyQTRZdU&dib_tag=se&keywords=playstation+4&qid=1719296832&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sr=8-2',
 '/PlayStation-Slim-1TB-Console-Spider-Man-4/dp/B07HHW8C4V/ref=sr_1_3?dib=eyJ2IjoiMSJ9.4

In [23]:
new_soup.find("span",attrs={"id":'productTitle'}).text

'        BOKESUYA 2 Pack Wireless Controller for PS4,Wireless Remote Controller with 1000mAh Battery,Double Shock/Audio/Six-axis Motion Sensor,Compatible with Playstation 4/Slim/Pro       '

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# Function to extract Product Title
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id": 'productTitle'})
        title_value = title.text
        title_string = title_value.strip()
    except AttributeError:
        title_string = ""
    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={'id': 'priceblock_ourprice'}).string.strip()
    except AttributeError:
        try:
            price = soup.find("span", attrs={'id': 'priceblock_dealprice'}).string.strip()
        except:
            price = ""
    return price

# Function to extract Product Rating
def get_rating(soup):
    try:
        rating = soup.find("i", attrs={'class': 'a-icon a-icon-star a-star-4-5'}).string.strip()
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class': 'a-icon-alt'}).string.strip()
        except:
            rating = ""
    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id': 'acrCustomerReviewText'}).string.strip()
    except AttributeError:
        review_count = ""
    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id': 'availability'})
        available = available.find("span").string.strip()
    except AttributeError:
        available = "Not Available"
    return available

if __name__ == '__main__':
    HEADERS = ({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    })

    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")

    links = soup.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    links_list = []
    for link in links:
        links_list.append(link.get('href'))

    d = {"title": [], "price": [], "rating": [], "reviews": [], "availability": []}

    for link in links_list:
        full_url = "https://www.amazon.com" + link
        try:
            new_webpage = requests.get(full_url, headers=HEADERS)
            new_webpage.raise_for_status()  # Check for request errors
            new_soup = BeautifulSoup(new_webpage.content, "html.parser")

            d['title'].append(get_title(new_soup))
            d['price'].append(get_price(new_soup))
            d['rating'].append(get_rating(new_soup))
            d['reviews'].append(get_review_count(new_soup))
            d['availability'].append(get_availability(new_soup))
        except requests.exceptions.RequestException as e:
            print(f"Skipping URL {full_url} due to {e}")

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [ ]:
amazon_df